# Process multiple ROI results


The multiple ROI analysis produces a n*n matrix detailing the number of waypoints between each ROI. To make these numbers more meaningful, you can rescale the results in a couple of different ways. Its important to note that these methods are for scaling, the pattern of results should stay the same.

### The simplest way of doing this, the [Gschwind method](https://academic.oup.com/cercor/article/22/7/1564/291933):

"... fiber tracking was initiated in both directions (from seed to target and vice versa), and these values were subsequently averaged. To obtain a measure of connectivity probability between ROIs (analysis 2), we used this average number of streamlines per seed voxel reaching the target (Croxson et al. 2005), expressed as a proportion of all successful samples in all pairwise connections in both hemispheres (see also Croxson et al. 2005; Eickhoff et al. 2010)."

So take the average waytotal (n connections) for each ROI pairing in both directions. Then express as a proportion of the total waytotal of all connections. 

### A second more complex way of calculating it is the [Eickhoff method](https://pubmed.ncbi.nlm.nih.gov/20445067/):

1. Waytotal of the connection (avereaged both ways) / Summed way total of seed to all other ROIs

2. Multiplied by the mean total waytotal of all seeds and target combinations. They call this a connection density value.

3. Divide by the size of the target ROI.

4. Multiply by the mean size of all target ROIs.


This way has the advantage of correcting for both ROI mask sizes. 

In [12]:
# Import all libraries needed

import os
import pandas as pd
import numpy as np
import glob
import seaborn as sns
from matplotlib import pyplot as plt
import statistics

In [13]:
# Collect some useful path and file info. 

folder_path = '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/'
subject_folders = sorted(glob.glob(folder_path))
print(subject_folders)

subjects = []
for file in sorted(glob.glob(folder_path)):
    name = file.split(os.path.sep)[-3]
    subjects.append(name)
    
print(subjects)

['/cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-02/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-03/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-04/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-05/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-06/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-07/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-08/dwi/', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-09/dwi/']
['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09']


## Determine the number of voxels for each participants ROI masks.

Counts the total and creates a file for each subject.

In [14]:
# this needs to be in the same order as it was processed. 
#The folder of the results will be named in this order. This roi argument is used extensively elsewhere.
rois = 'bst','ncc','subic', 'ant_thal', 'ext_glob_pal'
bst_masks = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_BST_central.nii.gz'))
ncc_masks = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_accumbcore_all_accum_nuc.nii'))
subic_masks = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_subiculum_all.nii'))
hippocampus_masks = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_hippocampus.nii'))
caudate_masks = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_caudate_nucleus.nii'))
anterior_thalamus = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_anteromedial_thalamic_all_anterior_thalamic.nii'))
ext_glob_pal = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ROIS/sub-*_registered_labels_ext_GP.nii'))

list_of_masks = list(zip(bst_masks,ncc_masks,subic_masks, anterior_thalamus, ext_glob_pal))


def get_mask_voxel_count (list_of_masks,rois, subject_folders):    
    for counter,folder in enumerate(subject_folders):
        os.chdir(folder)
        for roi_counter, roi in enumerate(rois):
            cmd = 'fslstats ' + list_of_masks[counter][roi_counter] + ' -V > ' + roi + '_voxel_size'
            print(cmd)
            os.system(cmd)
        
get_mask_voxel_count(list_of_masks,rois,subject_folders)  

fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/ROIS/sub-01_registered_labels_BST_central.nii.gz -V > bst_voxel_size
fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/ROIS/sub-01_registered_labels_accumbcore_all_accum_nuc.nii -V > ncc_voxel_size
fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/ROIS/sub-01_registered_labels_subiculum_all.nii -V > subic_voxel_size
fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/ROIS/sub-01_registered_labels_anteromedial_thalamic_all_anterior_thalamic.nii -V > ant_thal_voxel_size
fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/ROIS/sub-01_registered_labels_ext_GP.nii -V > ext_glob_pal_voxel_size
fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-02/dwi/ROIS/sub-02_registered_labels_BST_central.nii.gz -V > bst_voxel_size
fslstats /cubric/data/c1639425/Monkey_Brains/derivatives/sub-02/dwi/ROIS/sub-02_registered_labels_accumbcore_all_accum_nuc.nii -V > 

## Create DF of ROI voxel sizes. 

This also calculates the mean size of each seeds target ROIS, which is used later in some scaling methods.

In [15]:
# This is used later in some scaling methods

# Collect the number of voxels for each ROI you have used.
bst_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/bst_voxel_size'))
ncc_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ncc_voxel_size'))
subic_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/subic_voxel_size'))
hippocampus_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/hippocampus_voxel_size'))
caudate_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/caudate_voxel_size'))
anterior_thalamus_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ant_thal_voxel_size'))
ext_glob_pal_mask_voxels = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/ext_glob_pal_voxel_size'))
print(ext_glob_pal_mask_voxels)

list_of_mask_voxels = list(zip(bst_mask_voxels,ncc_mask_voxels,subic_mask_voxels,anterior_thalamus_mask_voxels, ext_glob_pal_mask_voxels))
#print(list_of_mask_voxels)

# Create dataframes. I was before also multiplying the ROI size * streamlines. This part has been commented out.

#ROI_total_size_mult_streamlines_df = pd.DataFrame()
#ROI_total_size_mult_streamlines_df['Subjects'] = subjects
ROI_total_size_df = pd.DataFrame()
ROI_total_size_df['Subjects'] = subjects

def create_mask_vox_df(subject_folders, rois, list_of_mask_voxels,ROI_total_size_df):
    for r,roi in enumerate(rois):
        current_roi_voxels = list()
        current_roi_voxels_mult = list()
        for s,subj in enumerate(subject_folders):
            ## total number of streamlines = Number of voxels within in seed mask * number of streamlines you randomly seed from it (should be 5000)
            current_roi_voxels.append(np.loadtxt(list_of_mask_voxels[s][r])[0])
            #current_roi_voxels_mult.append(current_roi_voxels[s] * 5000)
            #print(current_roi_voxels_mult)
        #ROI_total_size_mult_streamlines_df[roi + '_total_streamlines'] = current_roi_voxels_mult
        ROI_total_size_df[roi + '_size'] = current_roi_voxels
        #print(total_streamlines_df)
    # This is to get the mean size of all the target ROIS of each seed. Used in calculations later.     
    for r,roi in enumerate(rois):
        calc_mean_df = ROI_total_size_df.drop(columns=[roi + '_size'])
        #print(calc_mean_df)
        ROI_total_size_df[roi + '_target_ROI_sizes'] = calc_mean_df.mean(axis = 1)
    return ROI_total_size_df


ROI_total_size_df = create_mask_vox_df(subject_folders, rois, list_of_mask_voxels,ROI_total_size_df)   

print(ROI_total_size_df)

['/cubric/data/c1639425/Monkey_Brains/derivatives/sub-01/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-02/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-03/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-04/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-05/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-06/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-07/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-08/dwi/ext_glob_pal_voxel_size', '/cubric/data/c1639425/Monkey_Brains/derivatives/sub-09/dwi/ext_glob_pal_voxel_size']
  Subjects  bst_size  ncc_size  subic_size  ant_thal_size  ext_glob_pal_size  \
0   sub-01      62.0     182.0       207.0           39.0              393.0   
1   sub-02      54.0     155.0       191.0           41.0        

# Read in connectivity matrices

Now read in the connectivity matrix (output of the multiple ROI bedpostx) for each subject, creating a list of matricies.

In [17]:
number_of_subjects = 9
connection_matricies = [[] for _ in range(number_of_subjects)]

# remember to change this to your new output folder
matrix_outputs = sorted(glob.glob('/cubric/data/c1639425/Monkey_Brains/derivatives/sub-*/dwi/probabilistic.bedpostX/bst_ncc_subic_antthal_extglobpal_distance_uncorrected/fdt_network_matrix'))

# Put each subjects output of the multiple ROI analysis into a list of matricies.
def create_list_of_matricies(number_of_subjects,matrix_outputs):
    for i in range(number_of_subjects):
        connection_matricies[i-1] = np.loadtxt(matrix_outputs[i]) 
        #connection_matricies[i-1] = np.matrix(connection_matricies[i-1])
        print(connection_matricies[i-1])
        
create_list_of_matricies(number_of_subjects,matrix_outputs)       

[[     0.  70981.  10872. 115199.  53847.]
 [164243.      0.  12944.  83415.  37837.]
 [  8028.   2694.      0.  17836.   2089.]
 [ 97440.  37369.  44016.      0.  14739.]
 [151445.  24608.   6637.  39189.      0.]]
[[     0.  35235.  23086.  81964.  94288.]
 [ 47388.      0.  11101.  17010.  48853.]
 [ 19569.   3516.      0.  25282.   5639.]
 [ 54688.   6800.  55258.      0.  17501.]
 [191562.  30576.  11191.  42928.      0.]]
[[     0.  86070.  19152.  74317.  15744.]
 [259002.      0.  34111. 134300.  22867.]
 [ 21077.   6310.      0.   7377.  27262.]
 [ 81118.  49299.   5401.      0.   4552.]
 [ 58850.  42865.  44420.  22000.      0.]]
[[0.00000e+00 2.93100e+04 6.18000e+02 8.27200e+03 4.05530e+04]
 [1.82965e+05 0.00000e+00 1.46000e+02 1.38800e+03 1.34905e+05]
 [1.48100e+03 5.82000e+02 0.00000e+00 2.07000e+02 1.38500e+03]
 [3.07590e+04 1.73100e+03 6.28000e+02 0.00000e+00 1.22440e+04]
 [4.12940e+04 9.39380e+04 1.45600e+03 1.01430e+04 0.00000e+00]]
[[    0. 71527.  1623. 70128. 24750.

## Put matrices into a dataframe and collect useful stats along the way

This loop reads in the matricies and saves and calculates data for use in the normalisation calculations later on. 

In [18]:
# Go through the list of matricies and create a column for each connection with a row for every subject. Avereage 
# the connection each way (e.g amyg - hippocampus, hippocampus - amyg).

#Create some vars for the loop below

number_of_rois = len(rois)
# Where the avereaged waytotals go
connections_df = pd.DataFrame()
connections_df['Subjects'] = subjects
# Where the the average of waytotal for a connection in both directions, corrected by seed ROI sizes go.
connections_df_rois_size_corrected = pd.DataFrame()
connections_df_rois_size_corrected['Subjects'] = subjects
# Where the non-avereaged waytotals go for each connection. So just each seed to target.
all_non_averaged_connections = pd.DataFrame()
all_non_averaged_connections['Subjects'] = subjects
# The sum of the waytotals for a seed to each of its targets
total_seed_to_all_other_ROIS = pd.DataFrame()
total_seed_to_all_other_ROIS['Subjects'] = subjects
# The mean of all of the seed-target combinations
mean_of_all_connections = pd.DataFrame()
mean_of_all_connections['Subjects'] = subjects

def construct_connections_dfs(rois, ROI_total_size_df):
    for row in range(number_of_rois):
        current_seed = rois[row]
        # DF to collect each seeds results for each of its targets.
        target_waypoint_collector = pd.DataFrame()
        target_waypoint_collector['Subjects'] = subjects        
        for col in range(number_of_rois):
            if col == row:
                continue
            else:
                current_target = rois[col]
                #print('Current seed is ' + current_seed)
                #print('Current target is ' + current_target)
                current_connection_rois = rois[row] + '_' + rois[col]
                seed_roi_size = list(ROI_total_size_df[rois[row] + '_size'])
                target_roi_size = list(ROI_total_size_df[rois[col] + '_size'])
                # Collec the seed to target waytotal
                curr_connection = list()
                # Collect the target to seed waytotal (same rois, but in the opposite direction).
                avrg_with = list()
                for subj in range(len((subjects))):
                    curr_connection.append(connection_matricies[subj][row][col])
                    # Avereage with the same connection in the opposite direction.
                    avrg_with.append(connection_matricies[subj][col][row])
                # Put the waytotal of the connection (one way) into a df
                all_non_averaged_connections[current_connection_rois] = curr_connection
                # Correct the waytotal of the current connection by the size of the seed roi
                corrected_values = [i / j for i, j in zip(curr_connection, seed_roi_size)]
                # Correct the waytotal of the same connection in the opposite direction by its seed roi.
                avrg_with_corrected_values = [i / j for i, j in zip(avrg_with, target_roi_size)]
                # Take the mean of the connection in both directions (not avereaged by ROI size)
                mean_connection = [statistics.mean(k) for k in zip(curr_connection, avrg_with)]
                # Take the mean of the connection in both directions, accounting for ROI sizes.
                mean_connection_roi_size_corrected  = [statistics.mean(k) for k in zip(corrected_values,avrg_with_corrected_values)]
                connections_df[current_connection_rois] = mean_connection
                connections_df_rois_size_corrected[current_connection_rois] = mean_connection_roi_size_corrected
                #print(connections_df)
                # For each target, append into a list the total number of connections to the seed
                target_waypoint_collector[current_connection_rois] = curr_connection        
        #For each seed, sum the total of all connections to all targets.
        #print(target_waypoint_collector)
        target_waypoint_collector.drop(columns=['Subjects'])
        total_seed_to_all_other_ROIS[current_seed + '_total'] = target_waypoint_collector.sum(axis = 1)
        #print(total_seed_to_all_other_ROIS)
                

construct_connections_dfs(rois, ROI_total_size_df)            

print('The df with averaged connections is ')
display(connections_df)
print('The df with averaged connections, corrected for ROI size is ')
display(connections_df_rois_size_corrected)
print('The df with all connections, not averaged is ')
display(all_non_averaged_connections)

The df with averaged connections is 


,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_bst,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_bst,...,subic_ant_thal,subic_ext_glob_pal,ant_thal_bst,ant_thal_ncc,ant_thal_subic,ant_thal_ext_glob_pal,ext_glob_pal_bst,ext_glob_pal_ncc,ext_glob_pal_subic,ext_glob_pal_ant_thal
0,sub-01,41311.5,21327.5,68326.0,142925.0,41311.5,7308.5,11905.0,39714.5,21327.5,...,40270.0,8415.0,68326.0,11905.0,40270.0,30214.5,142925.0,39714.5,8415.0,30214.5
1,sub-02,172536.0,20114.5,77717.5,37297.0,172536.0,20210.5,91799.5,32866.0,20114.5,...,6389.0,35841.0,77717.5,91799.5,6389.0,13276.0,37297.0,32866.0,35841.0,13276.0
2,sub-03,106137.5,1049.5,19515.5,40923.5,106137.5,364.0,1559.5,114421.5,1049.5,...,417.5,1420.5,19515.5,1559.5,417.5,11193.5,40923.5,114421.5,1420.5,11193.5
3,sub-04,79055.0,1555.5,62815.0,32978.0,79055.0,489.0,13339.5,11390.0,1555.5,...,3175.5,306.0,62815.0,13339.5,3175.5,8472.0,32978.0,11390.0,306.0,8472.0
4,sub-05,74227.5,4941.5,67880.0,34747.5,74227.5,2078.0,29923.0,4003.5,4941.5,...,16982.0,2059.0,67880.0,29923.0,16982.0,14874.5,34747.5,4003.5,2059.0,14874.5
5,sub-06,65104.0,9302.5,17732.0,31185.5,65104.0,2550.0,4798.5,6135.0,9302.5,...,36604.0,1904.5,17732.0,4798.5,36604.0,1855.0,31185.5,6135.0,1904.5,1855.0
6,sub-07,70090.5,436.0,66804.0,16093.5,70090.5,79.0,16182.0,10636.0,436.0,...,6986.5,264.5,66804.0,16182.0,6986.5,9310.5,16093.5,10636.0,264.5,9310.5
7,sub-08,102171.0,9367.0,65932.5,15987.0,102171.0,4415.5,27064.5,1295.0,9367.0,...,22914.5,1078.5,65932.5,27064.5,22914.5,981.0,15987.0,1295.0,1078.5,981.0
8,sub-09,117612.0,9450.0,106319.5,102646.0,117612.0,7819.0,60392.0,31222.5,9450.0,...,30926.0,4363.0,106319.5,60392.0,30926.0,26964.0,102646.0,31222.5,4363.0,26964.0


The df with averaged connections, corrected for ROI size is 


,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_bst,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_bst,...,subic_ant_thal,subic_ext_glob_pal,ant_thal_bst,ant_thal_ncc,ant_thal_subic,ant_thal_ext_glob_pal,ext_glob_pal_bst,ext_glob_pal_ncc,ext_glob_pal_subic,ext_glob_pal_ant_thal
0,sub-01,414.340039,233.445535,1362.128205,1004.104654,414.340039,38.990006,133.910256,173.112302,233.445535,...,769.503530,27.858686,1362.128205,133.910256,769.503530,278.987571,1004.104654,173.112302,27.858686,278.987571
1,sub-02,1632.434767,232.508726,1677.364273,233.352183,1632.434767,126.553808,1034.433124,137.551719,232.508726,...,85.177372,137.467683,1677.364273,1034.433124,85.177372,88.250290,233.352183,137.551719,137.467683,88.250290
2,sub-03,751.040929,8.681474,497.310028,396.746612,751.040929,1.754587,27.854853,491.361062,8.681474,...,9.203618,5.092396,497.310028,27.854853,9.203618,183.092831,396.746612,491.361062,5.092396,183.092831
3,sub-04,936.561340,19.288938,1599.341026,293.664553,936.561340,2.655400,235.725862,50.884591,19.288938,...,59.713696,1.102382,1599.341026,235.725862,59.713696,160.153153,293.664553,50.884591,1.102382,160.153153
4,sub-05,792.847732,52.679638,1408.512511,225.008435,792.847732,10.441193,380.586257,17.377320,52.679638,...,241.168224,7.213420,1408.512511,380.586257,241.168224,120.291576,225.008435,17.377320,7.213420,120.291576
5,sub-06,581.441087,63.999163,384.392339,233.869426,581.441087,12.564647,46.521711,26.293911,63.999163,...,367.417925,7.485664,384.392339,46.521711,367.417925,13.419608,233.869426,26.293911,7.485664,13.419608
6,sub-07,698.388889,4.796987,1448.590476,166.797410,698.388889,0.388105,133.506878,45.477304,4.796987,...,146.165676,0.778977,1448.590476,133.506878,146.165676,126.383208,166.797410,45.477304,0.778977,126.383208
7,sub-08,1295.166828,127.619831,1543.023994,139.170211,1295.166828,23.750495,352.483950,4.927702,127.619831,...,463.133013,3.641855,1543.023994,352.483950,463.133013,15.622726,139.170211,4.927702,3.641855,15.622726
8,sub-09,854.429080,89.499461,1922.057502,527.230340,854.429080,34.067365,621.552404,110.576909,89.499461,...,545.214703,10.854454,1922.057502,621.552404,545.214703,212.376399,527.230340,110.576909,10.854454,212.376399


The df with all connections, not averaged is 


,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_bst,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_bst,...,subic_ant_thal,subic_ext_glob_pal,ant_thal_bst,ant_thal_ncc,ant_thal_subic,ant_thal_ext_glob_pal,ext_glob_pal_bst,ext_glob_pal_ncc,ext_glob_pal_subic,ext_glob_pal_ant_thal
0,sub-01,35235.0,23086.0,81964.0,94288.0,47388.0,11101.0,17010.0,48853.0,19569.0,...,25282.0,5639.0,54688.0,6800.0,55258.0,17501.0,191562.0,30576.0,11191.0,42928.0
1,sub-02,86070.0,19152.0,74317.0,15744.0,259002.0,34111.0,134300.0,22867.0,21077.0,...,7377.0,27262.0,81118.0,49299.0,5401.0,4552.0,58850.0,42865.0,44420.0,22000.0
2,sub-03,29310.0,618.0,8272.0,40553.0,182965.0,146.0,1388.0,134905.0,1481.0,...,207.0,1385.0,30759.0,1731.0,628.0,12244.0,41294.0,93938.0,1456.0,10143.0
3,sub-04,71527.0,1623.0,70128.0,24750.0,86583.0,589.0,15147.0,13205.0,1488.0,...,3251.0,245.0,55502.0,11532.0,3100.0,8962.0,41206.0,9575.0,367.0,7982.0
4,sub-05,72076.0,5049.0,76152.0,20165.0,76379.0,2365.0,38703.0,5355.0,4834.0,...,19011.0,1979.0,59608.0,21143.0,14953.0,7054.0,49330.0,2652.0,2139.0,22695.0
5,sub-06,43952.0,3526.0,13281.0,23020.0,86256.0,1964.0,7442.0,8006.0,15079.0,...,54004.0,2487.0,22183.0,2155.0,19204.0,787.0,39351.0,4264.0,1322.0,2923.0
6,sub-07,61905.0,501.0,72465.0,18922.0,78276.0,89.0,28250.0,13517.0,371.0,...,4427.0,121.0,61143.0,4114.0,9546.0,7907.0,13265.0,7755.0,408.0,10714.0
7,sub-08,103985.0,11195.0,57258.0,11431.0,100357.0,7479.0,34555.0,1058.0,7539.0,...,12966.0,758.0,74607.0,19574.0,32863.0,1101.0,20543.0,1532.0,1399.0,861.0
8,sub-09,70981.0,10872.0,115199.0,53847.0,164243.0,12944.0,83415.0,37837.0,8028.0,...,17836.0,2089.0,97440.0,37369.0,44016.0,14739.0,151445.0,24608.0,6637.0,39189.0


In [19]:
#Only one seed, just testing this atm
#just_subic_seed_data = all_non_averaged_connections[['Subjects','subic_bst','subic_ncc','subic_hippocampus','subic_caudate','subic_ant_thal']]
#print(just_subic_seed_data)

#all_non_averaged_connections = just_subic_seed_data

# Find out what the summed number of all connections is for each subject
all_non_averaged_connections['Total'] = all_non_averaged_connections.sum(axis=1, skipna = True)



# Calculate the mean of all seed and target combinations
all_non_averaged_connections['Mean_total'] = all_non_averaged_connections.mean(axis = 1, skipna = True)
display(all_non_averaged_connections)

all_non_averaged_connections.to_csv('/cubric/data/c1639425/Monkey_Brains/results_df/distance_corrected_raw_data_bst_subic_ncc_ant_thal_extGP_df', index = False)

# Add the total column to the df with averaged connections.

connections_df['Total'] = all_non_averaged_connections['Total']
connections_df['Mean_total'] = all_non_averaged_connections['Mean_total']
display(connections_df)

,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_bst,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_bst,...,ant_thal_bst,ant_thal_ncc,ant_thal_subic,ant_thal_ext_glob_pal,ext_glob_pal_bst,ext_glob_pal_ncc,ext_glob_pal_subic,ext_glob_pal_ant_thal,Total,Mean_total
0,sub-01,35235.0,23086.0,81964.0,94288.0,47388.0,11101.0,17010.0,48853.0,19569.0,...,54688.0,6800.0,55258.0,17501.0,191562.0,30576.0,11191.0,42928.0,823435.0,78422.380952
1,sub-02,86070.0,19152.0,74317.0,15744.0,259002.0,34111.0,134300.0,22867.0,21077.0,...,81118.0,49299.0,5401.0,4552.0,58850.0,42865.0,44420.0,22000.0,1016094.0,96770.857143
2,sub-03,29310.0,618.0,8272.0,40553.0,182965.0,146.0,1388.0,134905.0,1481.0,...,30759.0,1731.0,628.0,12244.0,41294.0,93938.0,1456.0,10143.0,594005.0,56571.904762
3,sub-04,71527.0,1623.0,70128.0,24750.0,86583.0,589.0,15147.0,13205.0,1488.0,...,55502.0,11532.0,3100.0,8962.0,41206.0,9575.0,367.0,7982.0,427151.0,40681.047619
4,sub-05,72076.0,5049.0,76152.0,20165.0,76379.0,2365.0,38703.0,5355.0,4834.0,...,59608.0,21143.0,14953.0,7054.0,49330.0,2652.0,2139.0,22695.0,503433.0,47946.000000
5,sub-06,43952.0,3526.0,13281.0,23020.0,86256.0,1964.0,7442.0,8006.0,15079.0,...,22183.0,2155.0,19204.0,787.0,39351.0,4264.0,1322.0,2923.0,354342.0,33746.857143
6,sub-07,61905.0,501.0,72465.0,18922.0,78276.0,89.0,28250.0,13517.0,371.0,...,61143.0,4114.0,9546.0,7907.0,13265.0,7755.0,408.0,10714.0,393765.0,37501.428571
7,sub-08,103985.0,11195.0,57258.0,11431.0,100357.0,7479.0,34555.0,1058.0,7539.0,...,74607.0,19574.0,32863.0,1101.0,20543.0,1532.0,1399.0,861.0,502413.0,47848.857143
8,sub-09,70981.0,10872.0,115199.0,53847.0,164243.0,12944.0,83415.0,37837.0,8028.0,...,97440.0,37369.0,44016.0,14739.0,151445.0,24608.0,6637.0,39189.0,995428.0,94802.666667


,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_bst,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_bst,...,ant_thal_bst,ant_thal_ncc,ant_thal_subic,ant_thal_ext_glob_pal,ext_glob_pal_bst,ext_glob_pal_ncc,ext_glob_pal_subic,ext_glob_pal_ant_thal,Total,Mean_total
0,sub-01,41311.5,21327.5,68326.0,142925.0,41311.5,7308.5,11905.0,39714.5,21327.5,...,68326.0,11905.0,40270.0,30214.5,142925.0,39714.5,8415.0,30214.5,823435.0,78422.380952
1,sub-02,172536.0,20114.5,77717.5,37297.0,172536.0,20210.5,91799.5,32866.0,20114.5,...,77717.5,91799.5,6389.0,13276.0,37297.0,32866.0,35841.0,13276.0,1016094.0,96770.857143
2,sub-03,106137.5,1049.5,19515.5,40923.5,106137.5,364.0,1559.5,114421.5,1049.5,...,19515.5,1559.5,417.5,11193.5,40923.5,114421.5,1420.5,11193.5,594005.0,56571.904762
3,sub-04,79055.0,1555.5,62815.0,32978.0,79055.0,489.0,13339.5,11390.0,1555.5,...,62815.0,13339.5,3175.5,8472.0,32978.0,11390.0,306.0,8472.0,427151.0,40681.047619
4,sub-05,74227.5,4941.5,67880.0,34747.5,74227.5,2078.0,29923.0,4003.5,4941.5,...,67880.0,29923.0,16982.0,14874.5,34747.5,4003.5,2059.0,14874.5,503433.0,47946.000000
5,sub-06,65104.0,9302.5,17732.0,31185.5,65104.0,2550.0,4798.5,6135.0,9302.5,...,17732.0,4798.5,36604.0,1855.0,31185.5,6135.0,1904.5,1855.0,354342.0,33746.857143
6,sub-07,70090.5,436.0,66804.0,16093.5,70090.5,79.0,16182.0,10636.0,436.0,...,66804.0,16182.0,6986.5,9310.5,16093.5,10636.0,264.5,9310.5,393765.0,37501.428571
7,sub-08,102171.0,9367.0,65932.5,15987.0,102171.0,4415.5,27064.5,1295.0,9367.0,...,65932.5,27064.5,22914.5,981.0,15987.0,1295.0,1078.5,981.0,502413.0,47848.857143
8,sub-09,117612.0,9450.0,106319.5,102646.0,117612.0,7819.0,60392.0,31222.5,9450.0,...,106319.5,60392.0,30926.0,26964.0,102646.0,31222.5,4363.0,26964.0,995428.0,94802.666667


# Eickhoff method

1) Waytotal of the connection (avereaged both ways)

2) Divide by summed way total of seed to all other ROIs

3) Multiplied by the mean total waytotal of all seeds and target combinations. They call this a connection density value.

4) Divide by the size of the target ROI.

5) Multiply by the mean size of all target ROIs.



In [20]:
eickhoff_df = pd.DataFrame()
eickhoff_df['Subjects'] = subjects

number_of_seeds = 5
def get_eickhoff_df():    
    for row in range(number_of_seeds):
        #row = 2 # for selecting a particular seed 
        current_seed = rois[row]
        for col in range(number_of_rois):
            if col == row:
                continue
            else:
                current_target = rois[col]
                print('The current seed ROI is ' + current_seed)
                print('The current target ROI is ' + current_target)
                current_connection_rois = rois[row] + '_' + rois[col]
                seed_roi_size = list(ROI_total_size_df[rois[row] + '_size'])
                target_roi_size = list(ROI_total_size_df[rois[col] + '_size'])
                curr_connection = list()
                avrg_with = list()
                for subj in range(len((subjects))):
                    curr_connection.append(connection_matricies[subj][row][col])
                    # For averaging with the same connection in the opposite direction.
                    avrg_with.append(connection_matricies[subj][col][row])
                # Take the mean of the connection in both directions
                #step_one = [statistics.mean(k) for k in zip(curr_connection, avrg_with)]
                print(seed_roi_size)
                streamlines_mult_seed_roi = [i * 5000 for i in seed_roi_size]
                print(streamlines_mult_seed_roi)
                step_one = [i / j for i,j in zip(curr_connection, streamlines_mult_seed_roi)]
                print('Mean of both connections is = ')
                print(step_one[0])
                # Divide by summed waytotal of the seed to all other ROIs
                print(total_seed_to_all_other_ROIS[current_seed + '_total'][0])
                step_two = [i / j for i, j in zip(step_one, total_seed_to_all_other_ROIS[current_seed + '_total'])]
                print(step_two[0])
                # Multiply by mean total of all seeds and target combos
                print(all_non_averaged_connections['Mean_total'][0])
                step_three = [i * j for i , j in zip(step_two,all_non_averaged_connections['Mean_total'])]
                print(step_three[0])
                # Divide by the size of the ROI target
                print(target_roi_size[0])
                step_four = [i / j for i, j in zip(step_three, target_roi_size)]
                print(step_four[0])
                # Multiply by mean size of all target ROIs
                print(ROI_total_size_df[current_seed + '_target_ROI_sizes'][0])
                step_five = [i * j for i, j in zip(step_four, ROI_total_size_df[current_seed + '_target_ROI_sizes'])]
                print(step_five[0])
                # Put into dataframe
                eickhoff_df[current_connection_rois] = step_five

                
get_eickhoff_df()
print(eickhoff_df)


The current seed ROI is bst
The current target ROI is ncc
[62.0, 54.0, 59.0, 52.0, 61.0, 62.0, 63.0, 51.0, 73.0]
[310000.0, 270000.0, 295000.0, 260000.0, 305000.0, 310000.0, 315000.0, 255000.0, 365000.0]
Mean of both connections is = 
0.11366129032258064
234573.0
4.845454946757753e-07
78422.38095238095
0.03799921137222353
182.0
0.00020878687567155784
205.25
0.04285350623158725
The current seed ROI is bst
The current target ROI is subic
[62.0, 54.0, 59.0, 52.0, 61.0, 62.0, 63.0, 51.0, 73.0]
[310000.0, 270000.0, 295000.0, 260000.0, 305000.0, 310000.0, 315000.0, 255000.0, 365000.0]
Mean of both connections is = 
0.07447096774193548
234573.0
3.1747459316262094e-07
78422.38095238095
0.024897113487701215
207.0
0.0001202759105686049
205.25
0.024686630644206155
The current seed ROI is bst
The current target ROI is ant_thal
[62.0, 54.0, 59.0, 52.0, 61.0, 62.0, 63.0, 51.0, 73.0]
[310000.0, 270000.0, 295000.0, 260000.0, 305000.0, 310000.0, 315000.0, 255000.0, 365000.0]
Mean of both connections is

In [21]:
#Save eickhoff df

eickhoff_df.to_csv('/cubric/data/c1639425/Monkey_Brains/results_df/proportion_eickhoff_streamline_corrected_bst_subic_ncc_ant_thal_extglobpal_df', index = False)

# Gschwind Method

Remove duplicate columns, then divide each value by the total number of connections for each participant.

In [23]:
# This function was downloaded from https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/

def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

connections_df_no_dupe = connections_df.drop(columns=getDuplicateColumns(connections_df))
connections_df_rois_size_corrected = connections_df_rois_size_corrected.drop(columns=getDuplicateColumns(connections_df_rois_size_corrected))
display(connections_df_no_dupe)
list(connections_df_no_dupe.columns) 
#display(connections_df_rois_size_corrected)

,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_ant_thal,subic_ext_glob_pal,ant_thal_ext_glob_pal,Total,Mean_total
0,sub-01,41311.5,21327.5,68326.0,142925.0,7308.5,11905.0,39714.5,40270.0,8415.0,30214.5,823435.0,78422.380952
1,sub-02,172536.0,20114.5,77717.5,37297.0,20210.5,91799.5,32866.0,6389.0,35841.0,13276.0,1016094.0,96770.857143
2,sub-03,106137.5,1049.5,19515.5,40923.5,364.0,1559.5,114421.5,417.5,1420.5,11193.5,594005.0,56571.904762
3,sub-04,79055.0,1555.5,62815.0,32978.0,489.0,13339.5,11390.0,3175.5,306.0,8472.0,427151.0,40681.047619
4,sub-05,74227.5,4941.5,67880.0,34747.5,2078.0,29923.0,4003.5,16982.0,2059.0,14874.5,503433.0,47946.000000
5,sub-06,65104.0,9302.5,17732.0,31185.5,2550.0,4798.5,6135.0,36604.0,1904.5,1855.0,354342.0,33746.857143
6,sub-07,70090.5,436.0,66804.0,16093.5,79.0,16182.0,10636.0,6986.5,264.5,9310.5,393765.0,37501.428571
7,sub-08,102171.0,9367.0,65932.5,15987.0,4415.5,27064.5,1295.0,22914.5,1078.5,981.0,502413.0,47848.857143
8,sub-09,117612.0,9450.0,106319.5,102646.0,7819.0,60392.0,31222.5,30926.0,4363.0,26964.0,995428.0,94802.666667


['Subjects',
 'bst_ncc',
 'bst_subic',
 'bst_ant_thal',
 'bst_ext_glob_pal',
 'ncc_subic',
 'ncc_ant_thal',
 'ncc_ext_glob_pal',
 'subic_ant_thal',
 'subic_ext_glob_pal',
 'ant_thal_ext_glob_pal',
 'Total',
 'Mean_total']

In [24]:
# Next step - divide each value by the total number of connections

proportion_gschwind_df = connections_df_no_dupe[['bst_ncc','bst_subic','bst_ant_thal','bst_ext_glob_pal','ncc_subic','ncc_ant_thal','ncc_ext_glob_pal','subic_ant_thal','subic_ext_glob_pal','ant_thal_ext_glob_pal']].div(connections_df_no_dupe.Total, axis=0)
proportion_gschwind_df.insert(0,'Subjects', subjects)
display(proportion_gschwind_df)



,Subjects,bst_ncc,bst_subic,bst_ant_thal,bst_ext_glob_pal,ncc_subic,ncc_ant_thal,ncc_ext_glob_pal,subic_ant_thal,subic_ext_glob_pal,ant_thal_ext_glob_pal
0,sub-01,0.050170,0.025901,0.082977,0.173572,0.008876,0.014458,0.048230,0.048905,0.010219,0.036693
1,sub-02,0.169803,0.019796,0.076487,0.036706,0.019890,0.090345,0.032345,0.006288,0.035273,0.013066
2,sub-03,0.178681,0.001767,0.032854,0.068894,0.000613,0.002625,0.192627,0.000703,0.002391,0.018844
3,sub-04,0.185075,0.003642,0.147056,0.077205,0.001145,0.031229,0.026665,0.007434,0.000716,0.019834
4,sub-05,0.147443,0.009816,0.134834,0.069021,0.004128,0.059438,0.007952,0.033732,0.004090,0.029546
5,sub-06,0.183732,0.026253,0.050042,0.088010,0.007196,0.013542,0.017314,0.103301,0.005375,0.005235
6,sub-07,0.178001,0.001107,0.169654,0.040871,0.000201,0.041096,0.027011,0.017743,0.000672,0.023645
7,sub-08,0.203361,0.018644,0.131232,0.031820,0.008789,0.053869,0.002578,0.045609,0.002147,0.001953
8,sub-09,0.118152,0.009493,0.106808,0.103117,0.007855,0.060669,0.031366,0.031068,0.004383,0.027088


In [25]:
# Save dataframe

proportion_gschwind_df.to_csv('/cubric/data/c1639425/Monkey_Brains/results_df/proportion_gschwind_bst_ncc_subic_antthal_globpal_df', index = False)